In [42]:
import torch
import torch.nn as nn
import netron

In [40]:
!pip install netron

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
You should consider upgrading via the '/home/matija/Luxonis/envs/base/bin/python -m pip install --upgrade pip' command.


## YOLOP

In [9]:
def autopad(k, p=None):  # kernel, padding
    # Pad to 'same'
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p


class Conv(nn.Module):
    # Standard convolution
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p), groups=g, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        try:
            self.act = Hardswish() if act else nn.Identity()
        except:
            self.act = nn.Identity()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))


class Bottleneck(nn.Module):
    # Standard bottleneck
    def __init__(self, c1, c2, shortcut=True, g=1, e=0.5):  # ch_in, ch_out, shortcut, groups, expansion
        super(Bottleneck, self).__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_, c2, 3, 1, g=g)
        self.add = shortcut and c1 == c2

    def forward(self, x):
        return x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))

class BottleneckCSP(nn.Module):
    # CSP Bottleneck https://github.com/WongKinYiu/CrossStagePartialNetworks
    def __init__(self, c1, c2, n=1, shortcut=True, g=1, e=0.5):  # ch_in, ch_out, number, shortcut, groups, expansion
        super(BottleneckCSP, self).__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = nn.Conv2d(c1, c_, 1, 1, bias=False)
        self.cv3 = nn.Conv2d(c_, c_, 1, 1, bias=False)
        self.cv4 = Conv(2 * c_, c2, 1, 1)
        self.bn = nn.BatchNorm2d(2 * c_)  # applied to cat(cv2, cv3)
        self.act = nn.LeakyReLU(0.1, inplace=True)
        self.m = nn.Sequential(*[Bottleneck(c_, c_, shortcut, g, e=1.0) for _ in range(n)])

    def forward(self, x):
        y1 = self.cv3(self.m(self.cv1(x)))
        y2 = self.cv2(x)
        return self.cv4(self.act(self.bn(torch.cat((y1, y2), dim=1))))

## YoloV5

In [3]:
class C3(nn.Module):
    # CSP Bottleneck with 3 convolutions
    def __init__(self, c1, c2, n=1, shortcut=True, g=1, e=0.5):  # ch_in, ch_out, number, shortcut, groups, expansion
        super().__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c1, c_, 1, 1)
        self.cv3 = Conv(2 * c_, c2, 1)  # optional act=FReLU(c2)
        self.m = nn.Sequential(*(Bottleneck(c_, c_, shortcut, g, e=1.0) for _ in range(n)))
        # self.m = nn.Sequential(*(CrossConv(c_, c_, 3, 1, g, 1.0, shortcut) for _ in range(n)))

    def forward(self, x):
        return self.cv3(torch.cat((self.m(self.cv1(x)), self.cv2(x)), 1))

## YoloV4-tiny

In [18]:
class CSPTiny(nn.Module):
    # CSP Bottleneck https://github.com/WongKinYiu/CrossStagePartialNetworks
    def __init__(self, c1):  # ch_in, ch_out, number, shortcut, groups, expansion
        super(CSPTiny, self).__init__()
        self.cv1 = Conv(c1, c1, 3, 1, p = 1)
        self.cv2 = Conv(c1, c1//2, 3, 1, p = 1)
        self.cv3 = Conv(c1//2, c1//2, 3, 1, p = 1)
        self.cv4 = Conv(c1, c1, 1, 1)

    def forward(self, x):
        y1 = self.cv1(x)
        y2 = self.cv2(y1)
        y3 = self.cv3(y2)
        y3 = torch.cat([y2, y3], dim = 1)
        y4 = self.cv4(y3)
        x = torch.cat([y1, y4], dim = 1)
        
        return x

In [37]:
class CSPTinyBottleneck(nn.Module):
    # CSP Bottleneck https://github.com/WongKinYiu/CrossStagePartialNetworks
    def __init__(self, c1):  # ch_in, ch_out, number, shortcut, groups, expansion
        super(CSPTinyBottleneck, self).__init__()
        self.cv1 = Bottleneck(c1, c1)
        self.cv2 = Bottleneck(c1, c1//2)
        self.cv3 = Bottleneck(c1//2, c1//2)
        self.cv4 = Conv(c1, c1, 1, 1)

    def forward(self, x):
        y1 = self.cv1(x)
        y2 = self.cv2(y1)
        y3 = self.cv3(y2)
        y3 = torch.cat([y2, y3], dim = 1)
        y4 = self.cv4(y3)
        x = torch.cat([y1, y4], dim = 1)
        
        return x

In [5]:
## Analysis

In [11]:
from fvcore.nn import FlopCountAnalysis, flop_count_table

In [25]:
inp = torch.rand(1, 64, 128, 128)

In [44]:
model = nn.Sequential(Conv(64, 128, 3, 2),BottleneckCSP(128, 128))
flops = FlopCountAnalysis(model, inp)
print(flop_count_table(flops, max_depth = 1))
print(model(inp).size())

| module   | #parameters or shape   | #flops   |
|:---------|:-----------------------|:---------|
| model    | 0.153M                 | 0.633G   |
|  0       |  73.984K               |  0.305G  |
|  1       |  78.72K                |  0.328G  |
torch.Size([1, 128, 64, 64])


In [45]:
torch.onnx.export(model, inp, "BottleneckCSP.onnx")
netron.start('BottleneckCSP.onnx')

Serving 'BottleneckCSP.onnx' at http://localhost:8081


('localhost', 8081)

In [46]:
model = nn.Sequential(Conv(64, 128, 3, 2), C3(128, 128))
flops = FlopCountAnalysis(model, inp)
print(flop_count_table(flops, max_depth = 1))
print(model(inp).size())

| module   | #parameters or shape   | #flops   |
|:---------|:-----------------------|:---------|
| model    | 0.148M                 | 0.614G   |
|  0       |  73.984K               |  0.305G  |
|  1       |  74.496K               |  0.31G   |
torch.Size([1, 128, 64, 64])


In [47]:
torch.onnx.export(model, inp, "C3.onnx")
netron.start('C3.onnx')

Serving 'C3.onnx' at http://localhost:20340


('localhost', 20340)

In [48]:
model = nn.Sequential(CSPTiny(64), nn.MaxPool2d(2))
flops = FlopCountAnalysis(model, inp)
print(flop_count_table(flops, max_depth = 1))
print(model(inp).size())

| module   | #parameters or shape   | #flops   |
|:---------|:-----------------------|:---------|
| 0        | 68.992K                | 1.14G    |
|  cv1     |  36.992K               |  0.609G  |
|  cv2     |  18.496K               |  0.305G  |
|  cv3     |  9.28K                 |  0.154G  |
|  cv4     |  4.224K                |  72.352M |
torch.Size([1, 128, 64, 64])


In [49]:
torch.onnx.export(model, inp, "CSPTiny.onnx")
netron.start('CSPTiny.onnx')

Serving 'CSPTiny.onnx' at http://localhost:24126


('localhost', 24126)

In [50]:
model = nn.Sequential(CSPTinyBottleneck(64), nn.MaxPool2d(2))
flops = FlopCountAnalysis(model, inp)
print(flop_count_table(flops, max_depth = 1))
print(model(inp).size())

| module   | #parameters or shape   | #flops   |
|:---------|:-----------------------|:---------|
| 0        | 35.84K                 | 0.6G     |
|  cv1     |  20.672K               |  0.343G  |
|  cv2     |  5.728K                |  96.207M |
|  cv3     |  5.216K                |  87.818M |
|  cv4     |  4.224K                |  72.352M |
torch.Size([1, 128, 64, 64])


In [51]:
torch.onnx.export(model, inp, "CSPTinyBottleneck.onnx")
netron.start('CSPTinyBottleneck.onnx')

Serving 'CSPTinyBottleneck.onnx' at http://localhost:23065


('localhost', 23065)